<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/PydanticOutputParser_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparando el entorno del cuaderno**

---


Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic desde los secretos de Colab para fines de autenticación.

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs.

Comenta (#) las librerias y modelos que no desees usar. El uso de las API de OpenAI, Anthropic y DeepSeek es de pago. El resto son gratuitas y para usarlas basta con registrarse y generar una API Key.

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU


# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic

# Importamos para formatear mejor la salida
from IPython.display import Markdown, display
from pprint import pprint


In [ ]:
# Con DeepSeek con su API propia
# model=deepseek-reasoner para R1; model=deepseek-chat para V3

# translator_llm = ChatOpenAI(model="deepseek-reasoner" ,api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.3)
# accuracy_llm = ChatOpenAI(model="deepseek-reasoner" ,api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.1)
# fluency_llm = ChatOpenAI(model="deepseek-reasoner" ,api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.7)
# fixer_llm = ChatOpenAI(model="deepseek-reasoner" ,api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0)

# Con OpenAI

# translator_llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.3)
# accuracy_llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.1)
# fluency_llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.7)
# fixer_llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0)

# Con Deepseek R1 en Groq

translator_llm =  ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.3)
accuracy_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.1)
fluency_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.7)
fixer_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0)

In [ ]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryWithErrorOutputParser
from pydantic import BaseModel, Field
from typing import List

# Modelo Pydantic para respuesta simple
class TranslationOutput(BaseModel):
    translation: str = Field(description="The professional translated text")

# Modelo Pydantic para respuesta de un modelo corrector
class RespuestaAgente (BaseModel):
    translation: str = Field(description="Texto con la traduccion generada por el agente.")
    suggestions: List[str] = Field(description="Lista de suggestions encontradas en el texto.")


parser_translation = PydanticOutputParser(pydantic_object=TranslationOutput)
parser_agent = PydanticOutputParser(pydantic_object=RespuestaAgente)

# Parser que intenta corregir errores, podria implementarse tambien RetryWithErrorOutputParser.from_llm
# o try-except con retries o algun parser personalizado
# el llm para esto podria er alguno concretocon temperatura=0
fixing_parser_tranlation = OutputFixingParser.from_llm(parser=parser_translation, llm=fixer_llm)
fixing_parser_agent = OutputFixingParser.from_llm(parser=parser_agent, llm=fixer_llm)



In [ ]:
# 3. Construir el prompt template con variables explícitas
translator_prompt_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior legal translator specializing in Intellectual Property documents. "
        "Translate from {source_language} to {target_language} with:\n"
        "- Perfect accuracy\n"
        "- Legal terminology consistency\n"
        "- Publication-ready quality\n\n"
        "**Strict requirements:**\n"
        # "1. Output MUST be valid JSON with double quotes\n"
        "2. Follow EXACTLY this structure:\n{format_instructions}\n"
        "3. No additional text or explanations"
    )),
    ("human", (
        "Translate this legal text with absolute precision:\n"
        "{original_text}\n\n"
        # "Output ONLY the raw JSON without formatting:"
    ))
])


# Definir variables de entrada explícitamente
""""
El uso de .with_options(input_variables=...) es una decisión conservadora para:
1-Explicitación clara: Dejar documentado qué variables se esperan.
2-Prevención de errores: En casos donde LangChain no detecte automáticamente todas las variables del template.
3-Control de versiones: Garantizar compatibilidad si LangChain cambia su comportamiento de inferencia.
"""
# translator_prompt_template = translator_prompt_template.with_options(
#     input_variables=[
#         "source_language",
#         "target_language",
#         "original_text",
#         "format_instructions"
#     ]
# )

# Luego, al momento de formatear el prompt para su uso, se pasan todas las variables necesarias,
# incluyendo las instrucciones de formato obtenidas de 'parser_translation'
# formatted_prompt = translator_prompt_template.format(
#     source_language="English",          # o el idioma de origen que necesites
#     target_language="Spanish",           # o el idioma destino que necesites
#     original_text="Your legal text here",  # el texto legal a traducir
#     format_instructions=parser_translation.get_format_instructions()
# )

# Ahora 'formatted_prompt' contiene el prompt completo con todas las variables sustituidas.





# accuracy_reviewer_prompt_template = ChatPromptTemplate.from_messages([
#     ("system", (
#         "You are an Accuracy Reviewer specializing in {source_language} to {target_language} translations.\n"
#         "**Strict instructions:**\n"
#         "1. Follow EXACTLY this structure:\n"
#         "2. Return a JSON object with EXACTLY these fields:\n"
#         "   - 'translation': corrected translation text\n"
#         "   - 'suggestions': list of suggestions using format '- ERROR: [issue] -> SUGGESTION: [fix]'\n\n"
#         "3. Follow these rules:\n"
#         "- Correct only accuracy errors (mistranslations, omissions, untranslated text)\n"
#         "- Maintain original style and format\n"
#         "- If no changes, return original text and empty list\n"
#         "- Use exactly the specified JSON structure"
#     )),
#     ("human", (
#         "Original Text ({source_language}):\n"
#         "{original_text}\n\n"
#         "Current Translation ({target_language}):\n"
#         "{translation}\n\n"
#         "Provide corrections in the required JSON format."
#     ))
# ])

# # Configuración adicional para el parser
# accuracy_reviewer_prompt_template = accuracy_reviewer_prompt_template.partial(
#     format_instructions=parser_agent.get_format_instructions()
# )

accuracy_reviewer_prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        (
            "You are an Accuracy Reviewer specializing in {source_language} to {target_language} translations.\n"
            "**Strict instructions:**\n"
            "1. Follow EXACTLY this structure:\n"
            "2. Return a JSON object with EXACTLY these fields:\n"
            "   - 'translation': corrected translation text\n"
            "   - 'suggestions': list of suggestions using format '- ERROR: [issue] -> SUGGESTION: [fix]'\n\n"
            "3. Follow these rules:\n"
            "- Correct only accuracy errors (mistranslations, omissions, untranslated text)\n"
            "- Maintain original style and format\n"
            "- If no changes, return original text and empty list\n"
            "- Use exactly the specified JSON structure\n\n"
            "{format_instructions}"
        )
    ),
    (
        "human",
        (
            "Original Text ({source_language}):\n"
            "{original_text}\n\n"
            "Current Translation ({target_language}):\n"
            "{translation}\n\n"
            "Provide corrections in the required JSON format."
        )
    )
])


# Prompt del Revisor de Fluidez
# También pedimos que aplique sus mejoras directamente y devuelva el texto final que propone.
fluency_reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a Fluency Reviewer specializing in {source_language} to {target_language} translations.\n\n"
        "**Strict Requirements:**\n"
        "1. Follow EXACTLY this structure:\n"
            "Return a JSON object with EXACTLY these fields:\n"
            "   - 'translation': corrected translation text\n"
            "   - 'suggestions': list of suggestions using format '- ERROR: [issue] -> SUGGESTION: [fix]'\n\n"
        "2. Focus only on:\n"
        "- Grammar/spelling errors\n"
        "- Natural flow in {target_language}\n"
        "- Cultural adaptation\n"
        "- If no changes, return original text and empty list\n"
        "- Use exactly the specified JSON structure\n\n"
        "{format_instructions}"
    )),
    ("human", (
        "Review this translation for fluency:\n"

            "Original Text ({source_language}):\n"
            "{original_text}\n\n"
            "Current Translation ({target_language}):\n"
            "{translation}\n\n"
            "Provide corrections in the required JSON format."

        "Provide corrections in required JSON format:"
    ))
])

El modelo recibira un texto original, una traduccion y debe realizar una correccion de la traduccion.
Como resultado debe devolver una nueva traduccion y una lista con los problema encontrados en cierto formato

In [ ]:
from google.colab import files

# Solicitar al usuario que suba un fichero
uploaded = files.upload()

# Tomar el nombre del primer fichero subido
file_name = list(uploaded.keys())[0]

# Leer el contenido del fichero
with open(file_name, 'r', encoding='utf-8') as f:
    original_text = f.read()

Saving texttexttext.txt to texttexttext (1).txt


In [ ]:
translator_chain = translator_prompt_template | translator_llm | fixing_parser_tranlation
accuracy_reviewer_chain = accuracy_reviewer_prompt_template | accuracy_llm | fixing_parser_agent
fluency_reviewer_chain = fluency_reviewer_prompt | fluency_llm | fixing_parser_agent

In [ ]:
# Importamos para formatear mejor la salida
from IPython.display import Markdown, display

Ejecutando multiples cadenas secuencialmente

In [ ]:
result_translator = translator_chain.invoke({
        "source_language": "English",
        "target_language": "Spanish",
        "original_text": original_text,
        "format_instructions": parser_translation.get_format_instructions()
    })

# print(type(result_translator))
# print(result_translator)

display(Markdown("###🌍 Resultado del traductor"))
display(Markdown(result_translator.translation))
display(Markdown("---"))


result_accuracy_reviewer = accuracy_reviewer_chain.invoke({
        "source_language": "English",
        "target_language": "Spanish",
        "original_text": original_text,
        "translation": result_translator.translation,
        "format_instructions": parser_agent.get_format_instructions()
    })

# print(type(result_accuracy_reviewer))
# print(result_accuracy_reviewer)

display(Markdown("###📝 Resultados del Accuracy reviewer"))
for suggestion in result_accuracy_reviewer.suggestions:
    print(suggestion)
display(Markdown("*Traducción editada:*"))
display(Markdown(result_accuracy_reviewer.translation))
display(Markdown("---"))

result_fluency_reviewer = fluency_reviewer_chain.invoke({
        "source_language": "English",
        "target_language": "Spanish",
        "original_text": original_text,
        "translation": result_accuracy_reviewer.translation,
        "format_instructions": parser_agent.get_format_instructions()
    })

# print(type(result_fluency_reviewer))
# print(result_fluency_reviewer)

display(Markdown("###🎯 Resultados del Fluency_reviewer"))
for suggestion in result_fluency_reviewer.suggestions:
    print(suggestion)
display(Markdown("*Traducción editada:*"))
display(Markdown(result_fluency_reviewer.translation))
display(Markdown("---"))
display(Markdown("✅ Proceso finalizado con éxito"))

###🌍 Resultado del traductor

La búsqueda filosófica de la sabiduría implica formular preguntas generales y fundamentales. Con frecuencia, no conduce a respuestas directas, pero puede ayudar a una persona a comprender mejor el tema, examinar su vida, disipar la confusión y superar los prejuicios y las ideas autoengañadas asociadas con el sentido común. Por ejemplo, Sócrates afirmó que "la vida no examinada no merece la pena ser vivida" para destacar el papel de la indagación filosófica en la comprensión de la propia existencia. Y según Bertrand Russell, "el hombre que no tiene ninguna tintura de filosofía atraviesa la vida encarcelado en los prejuicios derivados del sentido común, en las creencias habituales de su época o de su nación, y en las convicciones que han crecido en su mente sin la cooperación o el consentimiento de su razón deliberada."

---

###📝 Resultados del Accuracy reviewer

- ERROR: 'tintura' puede ser confuso en el contexto filosófico. -> SUGGESTION: Usar 'vestigio' en lugar de 'tintura' para mayor claridad.
- ERROR: La estructura de la oración es muy larga y podría ser confusa. -> SUGGESTION: Dividir la oración en dos para mejorar la claridad sin perder el significado.


...

- ERROR: 'formular' sounds formal -> SUGGESTION: Replace with 'hacer' for natural flow
- ERROR: 'autoengañadas' -> SUGGESTION: Use 'autoengañosas' for better adjective agreement
- ERROR: 'asociadas con el sentido común' -> SUGGESTION: Use 'propias del sentido común' for smoother reading
- ERROR: 'no merece la pena' -> SUGGESTION: Replace with 'no vale la pena' for more natural expression
- ERROR: 'el hombre' -> SUGGESTION: Replace with 'el ser humano' for inclusivity
- ERROR: Redundant 'deliberada' -> SUGGESTION: Simplify to 'su razón' for conciseness


*Traducción editada:*

La búsqueda filosófica de la sabiduría implica hacer preguntas generales y fundamentales. Con frecuencia, no conduce a respuestas directas, pero puede ayudar a una persona a comprender mejor el tema, examinar su vida, disipar la confusión y superar los prejuicios y las ideas autoengañosas propias del sentido común. Por ejemplo, Sócrates afirmó que "la vida no examinada no vale la pena ser vivida" para destacar el papel de la indagación filosófica en la comprensión de la propia existencia. Y según Bertrand Russell, "el ser humano que no tiene ningún vestigio de filosofía atraviesa la vida encarcelado en los prejuicios derivados del sentido común, en las creencias habituales de su época o de su nación, y en las convicciones que han crecido en su mente sin la cooperación o el consentimiento de su razón deliberada."

---

✅ Proceso finalizado con éxito

In [ ]:
# Con una sola cadena

from langchain_core.runnables import RunnablePassthrough

full_chain = RunnablePassthrough.assign(translation=translator_chain) | accuracy_reviewer_chain

# Esto no puede ser
refull_chain =  translator_prompt_template | llm | parser_translation | accuracy_reviewer_prompt_template | llm | parser_agent


resultado_final = full_chain.invoke({
    "source_language": "English",
    "target_language": "Spanish",
    "original_text": original_text
})